In [ ]:
import gcsfs
import yaml

In [ ]:
fs = gcsfs.GCSFileSystem("pangeo-181919")

In [ ]:
def make_yaml_catalog(fs, root):
            
    d = {"sources" : dict()}
        
    for path in fs.ls(root):
                
        name = path.split("/")[-2]
                
        if fs.isdir(path):
            
            if fs.exists(path + ".zgroup"):
                                
                if "plugins" not in d:
                    
                    d["plugins"] = {"source" : [{"module" : "intake_xarray"}]}
                    
                d["sources"][name] = {"driver" : "zarr",
                                      "args" : {"urlpath" : "gs://%s" % path,
                                                "storage_options" : {"token" : "anon"},
                                                "consolidated" : False},
                                      "description" : "",
                                      "metadata" : dict()}
                
                if fs.exists(path + ".zmetadata"):
                    
                    d["sources"][name]["args"]["consolidated"] = True
                                      
            else:
                
                d["sources"][name] = {"args" : {"path" : "{{CATALOG_DIR}}/%s/catalog.yaml" % name},
                                      "description" : "",
                                      "driver" : "intake.catalog.local.YAMLFileCatalog",
                                      "metadata" : dict()}
                
                make_yaml_catalog(fs, path)

        elif ".nc" in name:
            
            print("%s is an nc file; still need to work out how to handle these" % path)
            
    ! mkdir -p {root[12:]}
        
    with open(root[12:] + "catalog.yaml", "w") as outfile:
        
        yaml.dump(d, outfile)
        
    print(root)
            
make_yaml_catalog(fs, "pangeo-data/tracmip/")